<a href="https://colab.research.google.com/github/wobias12/Nuxt/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TASK1 - Web Scraping**


In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
!scrapy startproject imdb_top250

New Scrapy project 'imdb_top250', using template directory '/usr/local/lib/python3.10/dist-packages/scrapy/templates/project', created in:
    /content/imdb_top250

You can start your first spider with:
    cd imdb_top250
    scrapy genspider example example.com


In [ ]:
!ls imdb_top250/

imdb_top250  scrapy.cfg


In [ ]:
%%writefile imdb_top250/imdb_top250/spiders/imdb_spider.py

#This code will only retrieve the information from the main website

import scrapy

class MovieSpider(scrapy.Spider):
    name = "movie_spider"
    start_urls = [
        'https://web.archive.org/web/20231021155922/https://www.imdb.com/chart/top/',
    ]

    def parse(self, response):

        for movie in response.css('div.ipc-metadata-list-summary-item__tc'):

            title = movie.css('h3.ipc-title__text::text').get()


            movie_metadata = movie.css('div.sc-c7e5f54-7.brlapf.cli-title-metadata span.sc-c7e5f54-8.hgjcbi.cli-title-metadata-item::text').getall()

            imdb_rating = movie.css('span.ipc-rating-star--imdb::attr(aria-label)').get()
            if imdb_rating:
                imdb_rating = imdb_rating.replace("IMDb rating: ", "")

            if len(movie_metadata) >= 3:
                year = movie_metadata[0]
                duration = movie_metadata[1]
                agerating = movie_metadata[2]

                yield {
                    'title': title,
                    'year': year,
                    'duration': duration,
                    'agerating': agerating,
                    'imdb_rating': imdb_rating
                }





Overwriting imdb_top250/imdb_top250/spiders/imdb_spider.py


In [ ]:
%%writefile imdb_top250/imdb_top250/spiders/imdb_spider.py

#This code will retrieve the informations from the main site and from the movie subpages

import scrapy
import re

class MovieSpider(scrapy.Spider):
    name = "movie_spider"
    start_urls = [
        'https://web.archive.org/web/20231021155922/https://www.imdb.com/chart/top/',
    ]

    def parse(self, response):

        for movie in response.css('div.ipc-metadata-list-summary-item__tc'):

            title = movie.css('h3.ipc-title__text::text').get()


            movie_metadata = movie.css('div.sc-c7e5f54-7.brlapf.cli-title-metadata span.sc-c7e5f54-8.hgjcbi.cli-title-metadata-item::text').getall()

            imdb_rating = movie.css('span.ipc-rating-star--imdb::attr(aria-label)').get()
            if imdb_rating:
                imdb_rating = imdb_rating.replace("IMDb rating: ", "")

            relative_url = movie.css('a.ipc-title-link-wrapper::attr(href)').get()
            imdb_id = None
            full_url = None
            if relative_url:

                match = re.search(r'/title/(tt\d{7})/', relative_url)
                if match:
                    imdb_id = match.group(1)

                full_url = response.urljoin(relative_url)

            if len(movie_metadata) >= 3:
                year = movie_metadata[0]
                duration = movie_metadata[1]
                agerating = movie_metadata[2]

                yield response.follow(
                full_url,
                callback=self.parse_movie_page,
                meta={
                    'title': title,
                    'year': year,
                    'duration': duration,
                    'agerating': agerating,
                    'rating': imdb_rating,
                    'imdb_id': imdb_id,
                    'full_url': full_url
                }
            )

    def parse_movie_subpage(self, response):

        relative_poster_url = response.css("a.ipc-lockup-overlay::attr(href)").re_first(r"/mediaviewer/[^/]+")
        poster_url = None
        if relative_poster_url:

            imdb_id = response.meta['imdb_id']
            poster_url = f"/title/{imdb_id}{relative_poster_url}"

        plot = response.xpath('//span[@data-testid="plot-xs_to_m"]/text()').get()

        directors = response.xpath('//li[a[contains(@href, "ref_=tt_cl_dr")]]/a/text()').getall()

        cast = response.xpath('//a[@data-testid="title-cast-item__actor"]/text()').getall()

        yield {
            'title': response.meta['title'],
            'year': response.meta['year'],
            'duration': response.meta['duration'],
            'agerating': response.meta['agerating'],
            'rating': response.meta['rating'],
            'imdb_id': response.meta['imdb_id'],
            'full_url': response.meta['full_url'],
            'poster_url': poster_url,
            'plot': plot,
            'directors': directors,
            'cast': cast
        }

Overwriting imdb_top250/imdb_top250/spiders/imdb_spider.py


In [ ]:
!scrapy runspider imdb_top250/imdb_top250/spiders/imdb_spider.py

2024-11-03 16:35:38 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-03 16:35:38 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-11-03 16:35:38 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on

**TASK2 - Information Retrieval System**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


with open('/content/sample_data/imdb-movies.jsonl') as file:
    movie_data = pd.read_json(file, lines=True)


movie_data['info_combined'] = movie_data.apply(
    lambda row: f"{row['title']} {row['plot']} {' '.join(map(str, row['directors']))} {' '.join(map(str, row['actors']))} {row['year']}",
    axis=1
)


lemmatizer = WordNetLemmatizer()

def advanced_prepare_text(text):
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.lower().split()
    lemmatized = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(lemmatized)


movie_data['info_prepared'] = movie_data['info_combined'].apply(advanced_prepare_text)


vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english')
count_matrix = vectorizer.fit_transform(movie_data['info_prepared'])


def find_movie(query):

    query_prepared = advanced_prepare_text(query)

    query_vector = vectorizer.transform([query_prepared])

    similarities = cosine_similarity(query_vector, count_matrix).flatten()

    best_match_index = similarities.argmax()

    return {
        'title': movie_data.iloc[best_match_index]['title'],
        'year': movie_data.iloc[best_match_index]['year']
    }

# Enter here your queries
queries = [
   # "Action movie about a naval aviator starring Tom Cruise",
   # "A movie where a man witnesses his parents' death and then learns the art of fighting to confront injustice",
   # "An animation where a cowboy doll feels threatened when a new spaceman action figure becomes the top toy in a boy's bedroom.",
   # "A film about a Scottish warrior starring Mel Gibson."
]


def display_result(result):
    print(f"--- Movie Recommendation ---\nTitle: {result['title']}\nRelease Year: {result['year']}\n")

for query in queries:
    result = find_movie(query)
    display_result(result)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


--- Movie Recommendation ---
Title: 25. Saving Private Ryan
Release Year: 1998

--- Movie Recommendation ---
Title: 128. Batman Begins
Release Year: 2005

--- Movie Recommendation ---
Title: 76. Toy Story
Release Year: 1995

--- Movie Recommendation ---
Title: 78. Braveheart
Release Year: 1995

--- Movie Recommendation ---
Title: 19. Se7en
Release Year: 1995

